# ControlNet

In [ ]:
%%capture
!pip install -qq opencv-contrib-python diffusers transformers git+https://github.com/huggingface/accelerate.git

In [ ]:
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

image = load_image("https://cdn-ak.f.st-hatena.com/images/fotolife/a/aa_debdeb/20160714/20160714112033.jpg")

In [ ]:
import cv2
from PIL import Image
import numpy as np

low_threshold = 100
high_threshold = 200

np_image = np.array(image)

canny_image = cv2.Canny(np_image, low_threshold, high_threshold)

canny_image = canny_image[:, :, None]
canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
canny_image = Image.fromarray(canny_image)

canny_image

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
import torch

# Canny Edge用のControlNetを指定　（このモデルは固定）
controlnet_model = "lllyasviel/sd-controlnet-canny"

# 好きなStable Diffusionを使用 (e.g. huggingface.coとかcivitai.com)
sd_model = "runwayml/stable-diffusion-v1-5"

# ControlNetモデルのダウンロード
controlnet = ControlNetModel.from_pretrained(
    controlnet_model,
    torch_dtype=torch.float16
)

# Stable Diffusion & ControlNet パイプラインの構築
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_model,
    controlnet=controlnet,
    torch_dtype=torch.float16
)

# 多く試してはいないが、スケジューラーによって結果が変わった
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
prompt = "wonder woman in DC comics"
num_steps = 20
seed = 0

out_image = pipe(
    prompt,
    num_inference_steps=num_steps,
    generator=torch.manual_seed(seed),
    image=canny_image
).images[0]

out_image

In [ ]:
print(f"prompt: {prompt}")

Image.fromarray(np.concatenate([image, canny_image, out_image], axis=1))

In [ ]:
from transformers import pipeline
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from PIL import Image
import numpy as np
import torch
from diffusers.utils import load_image

# Depth-to-Imageではdepth-estimationというモデルを追加でロード
depth_estimator = pipeline('depth-estimation')

In [ ]:
image = load_image("https://i.imgur.com/2V9gTol.jpg")
# image = load_image("http://file.mk.co.kr/meet/neds/2021/06/image_readtop_2021_583221_16238253014683484.jpg")

image

In [ ]:
depth_image = depth_estimator(image)['depth']
depth_image = np.array(depth_image)
depth_image = depth_image[:, :, None]
depth_image = np.concatenate([depth_image, depth_image, depth_image], axis=2)
depth_image = Image.fromarray(depth_image)

depth_image

In [ ]:
# Depth用のControlNetを指定　（このモデルは固定）
controlnet_model = "fusing/stable-diffusion-v1-5-controlnet-depth"

# アニメ風にするモデル (https://huggingface.co/andite/anything-v4.0)
sd_model = "andite/anything-v4.0"

controlnet = ControlNetModel.from_pretrained(
    controlnet_model,
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_model,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16,
)

pipe.enable_model_cpu_offload()

In [ ]:
# 他のスケジューラーも使ってみましょう
# 作ったpipeに使用できるスケジューラーを表示
pipe.scheduler.compatibles

In [ ]:
from diffusers import DDIMScheduler, DPMSolverMultistepScheduler, HeunDiscreteScheduler, DDPMScheduler, EulerDiscreteScheduler, KDPM2DiscreteScheduler, EulerAncestralDiscreteScheduler, DEISMultistepScheduler, KDPM2AncestralDiscreteScheduler, UniPCMultistepScheduler, LMSDiscreteScheduler, PNDMScheduler, DPMSolverSinglestepScheduler

pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

In [ ]:
prompt = "scenery, mountain, village, outdoors, sky, clouds"

num_steps = 50
guidance_scale = 14
seed = 1

out_image = pipe(
    prompt,
    num_inference_steps=num_steps,
    guidance_scale=guidance_scale,
    generator=torch.manual_seed(seed),
    image=depth_image,
).images[0]

out_image

In [ ]:
print(f"prompt: {prompt}")

Image.fromarray(np.concatenate([image.resize(out_image.size), depth_image.resize(out_image.size), out_image], axis=1))

In [ ]:
!pip install -qq controlnet_aux

In [ ]:
from PIL import Image
import numpy as np
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

openpose = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')

In [ ]:
image = load_image("https://i.imgur.com/uJpMDyu.jpg")

image

In [ ]:
# Check Pointなし(non-ema)のStable Diffusionモデルをダウンロード
!git clone -b non-ema --single-branch https://huggingface.co/runwayml/stable-diffusion-v1-5

In [ ]:
# civitaiからweightだけをダウンロードしてStable Diffusionモデルの配下に保存
!wget https://civitai.com/api/download/models/7974 -O stable-diffusion-v1-5/etherRealMixERM_etherrealmixERMV1.safetensors

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
# Open-Pose用のControlNetを指定（このモデルは固定）
controlnet_model = "fusing/stable-diffusion-v1-5-controlnet-openpose"

# Stable Diffusionモデルになっているが、実際に使うのはダウンロードしたEther Real Mix - ERMモデル
sd_model = "/content/stable-diffusion-v1-5"

controlnet = ControlNetModel.from_pretrained(
    controlnet_model,
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_model,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.enable_model_cpu_offload()

In [ ]:
pipe.scheduler.compatibles

In [ ]:
from diffusers import DDIMScheduler, DPMSolverMultistepScheduler, HeunDiscreteScheduler, DDPMScheduler, EulerDiscreteScheduler, KDPM2DiscreteScheduler, EulerAncestralDiscreteScheduler, DEISMultistepScheduler, KDPM2AncestralDiscreteScheduler, UniPCMultistepScheduler, LMSDiscreteScheduler, PNDMScheduler, DPMSolverSinglestepScheduler

pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

In [ ]:
#prompt = "RAW photo, a portrait photo of 50 y.o japanese man in clothes, night tokyo, (high detailed skin:1.2), 8k uhd, dslr, soft lighting, high quality, film grain, Fujifilm XT3"
prompt = "photo-realistic, may sometimes affect the quality if you give it a high weight, which makes the final picture looks like scattering glasses.omertosa,head "
negative_prompt = "NSFW, (worst quality, low quality:1.3), watermark, signature"
num_steps = 28
guidance_scale = 7
seed = 51251555

out_image = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_steps,
    guidance_scale=guidance_scale,
    generator=torch.manual_seed(seed),
    image=pose_image,
).images[0]

out_image

In [ ]:
print(f"prompt: {prompt}")

Image.fromarray(np.concatenate([image.resize(out_image.size), pose_image.resize(out_image.size), out_image], axis=1))

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, AutoencoderKL
from diffusers.utils import load_image
from diffusers import DDIMScheduler, DPMSolverMultistepScheduler, HeunDiscreteScheduler, DDPMScheduler, EulerDiscreteScheduler, KDPM2DiscreteScheduler, EulerAncestralDiscreteScheduler, DEISMultistepScheduler, KDPM2AncestralDiscreteScheduler, UniPCMultistepScheduler, LMSDiscreteScheduler, PNDMScheduler, DPMSolverSinglestepScheduler
import cv2
from PIL import Image
import numpy as np
from diffusers import DDPMScheduler
import torch
import random, sys

controlnet_model = "lllyasviel/sd-controlnet-canny"
sd_model = "Lykon/DreamShaper"

controlnet = ControlNetModel.from_pretrained(
    controlnet_model,
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    sd_model,
    controlnet=controlnet,
    torch_dtype=torch.float16
)

pipe.scheduler = PNDMScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def img2img(img_path, prompt, negative_prompt, num_steps=20, guidance_scale=7, seed=0, low=100, high=200):
    image = load_image(img_path)

    np_image = np.array(image)

    canny_image = cv2.Canny(np_image, low, high)

    canny_image = canny_image[:, :, None]
    canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
    canny_image = Image.fromarray(canny_image)

    out_image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=torch.manual_seed(seed),
        image=canny_image
    ).images[0]

    return image, canny_image, out_image

In [ ]:
prompt = "masterpiece, best quality, ultra-detailed, illustration, school uniform, scarf, gymnasium"
negative_prompt = "lowres, ((bad anatomy)), ((bad hands)), text, missing finger, extra digits, fewer digits, blurry, ((mutated hands and fingers)), (poorly drawn face), ((mutation)), ((deformed face)), (ugly), ((bad proportions)), ((extra limbs)), extra face, (double head), (extra head), ((extra feet)), monster, logo, cropped, worst quality, low quality, normal quality, jpeg, humpbacked, long body, long neck, ((jpeg artifacts))"
num_steps = 20
guidance_scale = 7
seed = 3467120481370323442

img_path = "https://post-phinf.pstatic.net/MjAyMzAzMDlfMjcy/MDAxNjc4MjkxNjU5MzE4.2VqU5CdXbJlPO7YIFkya7qnKIEsngQ1yfehnuZb0y_Eg.ZCAqiwDzQfSQ8_5PTPklTl9DFCRC3eUua_v3HfrvsyUg.PNG/image_8486401811678287934915.png?type=w1200"

image, canny_image, out_image = img2img(img_path, prompt, negative_prompt, num_steps, guidance_scale, seed, low=50, high=100)

out_image.save("02_result.png")
Image.fromarray(np.concatenate([image.resize(out_image.size), canny_image.resize(out_image.size), out_image], axis=1))

In [ ]:
prompt = "masterpiece, best quality, ultra-detailed, illustration, a man sitting, a lady sitting, tree, autumn, fallen leaves"
negative_prompt = "NSFW, lowres, ((bad anatomy)), ((bad hands)), text, missing finger, extra digits, fewer digits, blurry, ((mutated hands and fingers)), (poorly drawn face), ((mutation)), ((deformed face)), (ugly), ((bad proportions)), ((extra limbs)), extra face, (double head), (extra head), ((extra feet)), monster, logo, cropped, worst quality, low quality, normal quality, jpeg, humpbacked, long body, long neck, ((jpeg artifacts))"
num_steps = 20
guidance_scale = 7
seed = 3311352378018201782

img_path = "https://post-phinf.pstatic.net/MjAyMzAzMDlfMTI2/MDAxNjc4MjkxNjY4MDY1.VTeW8x873IE03L2xJi5NDr1HGskSA4Y3Zws_aN68DEsg.tsx75T60hL6ejyzMnzo7kxfj1VK_f0VzFONY56T0VtMg.JPEG/image_8366549961678289357607.jpg?type=w1200"

image, canny_image, out_image = img2img(img_path, prompt, negative_prompt, num_steps, guidance_scale, seed, 50, 80)

out_image.save("12_result.png")
Image.fromarray(np.concatenate([image.resize(out_image.size), canny_image.resize(out_image.size), out_image], axis=1))

In [ ]:
prompt = "masterpiece, best quality, ultra-detailed, illustration, a girl with backpack, a man down on his knee"
negative_prompt = "NSFW, lowres, ((bad anatomy)), ((bad hands)), text, missing finger, extra digits, fewer digits, blurry, ((mutated hands and fingers)), (poorly drawn face), ((mutation)), ((deformed face)), (ugly), ((bad proportions)), ((extra limbs)), extra face, (double head), (extra head), ((extra feet)), monster, logo, cropped, worst quality, low quality, normal quality, jpeg, humpbacked, long body, long neck, ((jpeg artifacts))"
num_steps = 20
guidance_scale = 7
seed = 3465640518011203857

img_path = "https://post-phinf.pstatic.net/MjAyMzAzMDlfMjQg/MDAxNjc4Mjk0MjE1MTMy.IQ5DuY2RIoErY_o-wA73wdWyVgMM8VtbkWgDI5nyw_8g.9lJrV2mwz_yvPOFSKlb6Hg_TWYrpryTTtwxiTDx08uUg.JPEG/image_888375631678289902427.jpg?type=w1200"

image, canny_image, out_image = img2img(img_path, prompt, negative_prompt, num_steps, guidance_scale, seed, 50, 100)

out_image.save("14_result.png")
Image.fromarray(np.concatenate([image.resize(out_image.size), canny_image.resize(out_image.size), out_image], axis=1))